In [34]:
import pandas as pd
import numpy as np
import nltk
import ast #this model is used to convert string to list

In [35]:
movies = pd.read_csv('data/tmdb_5000_movies.csv')
credits = pd.read_csv('data/tmdb_5000_credits.csv')

In [36]:
movies = movies.merge(credits, on='title')

In [37]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [38]:
movies.dropna(inplace=True)

In [39]:
def convert(obj):
    l = []
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l

In [40]:
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)

In [41]:
def convert3(obj):
    l = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            l.append(i['name'])
            counter += 1
    return l

In [42]:
movies['cast'] = movies['cast'].apply(convert3)

In [43]:
def fetch_director(obj):
    l = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            l.append(i['name'])
            break
    return l

In [44]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [45]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [46]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ", "") for i in x])

In [47]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [48]:
new_df = movies[['movie_id','title','tags']]

In [49]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [50]:
def stem(text):
    y = []
    
    for i in text:
        y.append(ps.stem(i))
    
    return " ".join(y)

In [51]:
new_df['tags'] = new_df['tags'].apply(stem)

C:\Users\Dhruv Bhatada\AppData\Local\Temp\ipykernel_1252\3213734980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [52]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

C:\Users\Dhruv Bhatada\AppData\Local\Temp\ipykernel_1252\3214958533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


In [53]:
from sklearn.feature_extraction.text import CountVectorizer
cv =  CountVectorizer(max_features = 5000, stop_words='english')

In [54]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [55]:
from sklearn.metrics.pairwise import cosine_similarity

In [56]:
similarity = cosine_similarity(vectors)

e:\Dhruv\PRoject\intern\Movie-Recommender\myenv\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [57]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [58]:
import pickle

In [59]:
pickle.dump(new_df, open('movies.pkl','wb'))

In [60]:
new_df.to_csv('Cleaned_DF.csv')

In [61]:
pickle.dump(similarity, open('similarity.pkl','wb'))